In [111]:
%matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.special

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

from tensorflow.python.client import device_lib

device_lib.list_local_devices()

Using matplotlib backend: Qt5Agg


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10996398366477853953
 xla_global_id: -1]

In [166]:
def Func(x, y):
    z = (np.sin(x) * np.exp((1 - np.cos(y)) ** 2) + np.cos(y) * np.exp((1 - np.sin(x)) ** 2) + (x - y) ** 2)
    return z

In [167]:
class MyNN:
    def __init__(self, rate, inputs, hiddens, outputs):
        #добавляем 1 вход под bias
        self.i_count=inputs+1
        self.h_count=hiddens 
        self.o_count=outputs
        #заполняем массивы весов рандомом
        self.w_ih = np.random.normal(0.0, pow(self.h_count, -0.5), (self.h_count, self.i_count))
        self.w_ho = np.random.normal(0.0, pow(self.o_count, -0.5), (self.o_count, self.h_count))
        #learning rate и сигмоид 
        self.lr = rate
        self.activation_function = lambda x: scipy.special.expit(x)
        
    def train(self, inputs_list, targets_list):
        #добавляем 1 вход под bias
        inputs_list=np.concatenate((inputs_list,[1]), axis=0)
        #вектор-столбцы входных данных и правильных ответов
        inputs = np.array(inputs_list, ndmin=2).T 
        targets = np.array(targets_list, ndmin=2).T
        #прямое распространение. сигмоид и линеар.
        hid_results = self.activation_function(np.dot(self.w_ih, inputs))
        out_results = np.dot(self.w_ho, hid_results)
        
        #ошибки вывода
        out_errors = (targets - out_results)
        #ошибки скрытого слоя
        hid_errors = np.dot(self.w_ho.T, out_errors)
        #поправки для весов скрытый-выход
        self.w_ho += self.lr * np.dot(out_errors, np.transpose(hid_results))
        #поправки для весов вход-скрытый
        self.w_ih += self.lr * np.dot((hid_errors * hid_results * (1.0 - hid_results)), np.transpose(inputs))
        
    def query(self, inputs_list):
        #добавляем 1 вход под bias
        inputs_list=np.concatenate((inputs_list,[1]), axis=0)
        #вектор-столбец входных данных
        inputs = np.array(inputs_list, ndmin=2).T 
        #прямое распространение. сигмоид и линеар.
        hid_results = self.activation_function(np.dot(self.w_ih, inputs))
        out_results = np.dot(self.w_ho, hid_results)
        return out_results
    
    def set_lr(self, rate):
        self.lr = rate


In [172]:
myNN = MyNN(rate=0.0001, inputs=2, hiddens=100, outputs=1)

In [175]:
xy_set = []
z_set = []
d = 10

for x in range(-5*d, 5*d):
    for y in range(-5*d, 5*d):
        xy_set += [[x/d, y/d]]
        z_set += [Func(x/d, y/d)]
        
xy_set = np.array(xy_set)
z_set = np.array(z_set)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(xy_set[:, 0], xy_set[:, 1], z_set, c='blue', marker='o')

plt.pause(5)

In [174]:
myNN.set_lr(0.0001)
epochs = 100
plt.ion()
fig = plt.figure()
ax = fig.gca(projection='3d')   
scat1 = ax.scatter(xy_set[:,0], xy_set[:,1], z_set, c='blue', marker='o')  
scat2 = ax.scatter(xy_set[:,0], xy_set[:,1], z_set, c='blue', marker='o')  
for e in range(epochs):
    
    for kkk in range(10):
        for x in range(10000):
            myNN.train(xy_set[x,:], z_set[x])
        
    MSE=0
    test_results=np.zeros(10000)
    for x in range(10000):
        result = myNN.query(xy_set[x,:])
        MSE+= (z_set[x] - result)**2
        test_results[x]=result
    
    scat2.remove() 
    scat2=ax.scatter(xy_set[:,0], xy_set[:,1], test_results, c='red', marker='o')     
    plt.pause(5)
    print(MSE/10000)

[[1287.44683524]]
[[1055.36257321]]
[[1172.68695288]]
[[1213.06439677]]
[[1204.02789609]]
[[1050.88973577]]
[[937.123009]]
[[699.19068509]]
[[846.36615426]]
[[445.13215818]]
[[469.70819283]]
[[385.96499651]]
[[409.83627881]]
[[363.25222899]]
[[268.14269505]]
[[196.72109211]]
[[153.21773207]]
[[142.88901529]]
[[131.48945526]]
[[122.38002623]]
[[117.90160238]]
[[116.18502675]]
[[115.90217316]]
[[117.9333425]]
[[152.6267491]]
[[265.81359665]]
[[219.03697822]]
[[218.25732775]]
[[404.26771394]]
[[436.67762709]]
[[198.32323263]]
[[300.09077544]]
[[192.53040131]]
[[372.69476855]]
[[208.2150334]]
[[184.05721066]]
[[145.39394578]]
[[144.04429157]]
[[148.87894909]]
[[135.28342251]]
[[133.05060584]]
[[124.76656841]]
[[123.68514792]]
[[134.22137863]]
[[123.20893871]]
[[135.82926133]]
[[230.47699163]]
[[224.73635952]]
[[210.22908349]]
[[207.66172591]]
[[182.20769407]]
[[170.01378098]]
[[162.27796798]]
[[180.56228679]]
[[263.96135406]]
[[230.73544045]]
[[263.01504981]]
[[192.95832702]]
[[417.0315218